In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType

In [ ]:
neObjectData = "exam_ex2_data/NEObject.txt"
observationData = "exam_ex2_data/Observations.txt"

outputPath1 = "outPart1/"
outputPath2 = "outPart2/"

In [ ]:
# Define the dataframes associated with the input files
neosDF = spark.read.format("csv")\
                    .option("header", True)\
                    .option("inferSchema", True)\
                    .load(neObjectData)


# Register the temporary table NEOs 
neosDF.createOrReplaceTempView("NEOs");


observationsDF = spark.read.format("csv")\
                    .option("header", True)\
                    .option("inferSchema", True)\
                    .load(observationData)


# Register the temporary table Observations 
observationsDF.createOrReplaceTempView("Observations")

In [ ]:
######################
# PART 1
######################

In [ ]:
# Compute the average dimension considering all NEObjects
avgDimensionDF = spark.sql("""SELECT AVG(Dimension) as AvgDim """+
                                        """FROM NEOs""")
    
# Register the temporary table avgDimension
avgDimensionDF.createOrReplaceTempView("avgDimension")

In [ ]:
# Analyze the content of NEOs by filtering by dimension and alreadyFallen fields.
# The resulting RDD contains the Most Relevant NEOIDs
mostRelNEOsDF = spark.sql("""SELECT NEOID """
                          + """FROM NEOs, avgDimension """
                          + """WHERE Dimension>AvgDim """
                          + """  AND alreadyFallen=false""")\
                    .cache()

# Register the temporary table mostRelNEOs
mostRelNEOsDF.createOrReplaceTempView("mostRelNEOs")

In [ ]:
# Register a function that given a month returns the year
spark.udf.register("EXTRACTYEAR", \
                   lambda month: int(month.split("-")[0]),\
                   IntegerType())

In [ ]:
# Keep only Observations made in 2023 or later
# and select NEOID and ObservatoryID (the second field is used in the second part of this application).
observationsFrom2023DF = spark.sql("""SELECT NEOID, ObservatoryID """
                                   + """FROM Observations """
                                   + """WHERE EXTRACTYEAR(ObsDateTime)=2023""")\
                                .cache();

# Register the temporary table observationsFrom2023
observationsFrom2023DF.createOrReplaceTempView("observationsFrom2023")

In [ ]:
# Join observationsFrom2023 with mostRelNEOs, obtaining all the observations in 2023 or later 
# associated with the Most Relevant NEOIDs.
# Then, count the number of observations starting from 2023 for each Most Relevant NEOID.
# Finally, sort data by number of observations (descending)
observationsPerNeoIdDF = spark.sql("""SELECT mostRelNEOs.NEOID, COUNT(*) """
                                   + """FROM observationsFrom2023, mostRelNEOs """
                                   + """WHERE mostRelNEOs.NEOID=observationsFrom2023.NEOID """
                                   + """GROUP BY mostRelNEOs.NEOID """
                                   + """ORDER BY COUNT(*) DESC""")

In [ ]:
observationsPerNeoIdDF.show()

In [ ]:
observationsPerNeoIdDF.write\
                    .format("csv")\
                    .option("header", False)\
                    .save(outputPath1)

In [ ]:
######################
# PART 2
######################

In [ ]:
# To compute the most relevant NEOIDs observed by less than 10 distinct observatories, 
# we can reuse the previously computed tables
# observationsFrom2023: contains the observatories per each NEOID
# mostRelNEOs: contains the Most Relevant NEOIDs

In [ ]:
# Perform a distinct over observationsFrom2023 to find the distinct observatories per NEOID,
# starting from 2023
distinctObservatoriesPerNeoIdDF = spark.sql("""SELECT DISTINCT * """
                                            + """FROM observationsFrom2023""")\
                                        .cache()
        
# Register the temporary table distinctObservatoriesPerNeoId
distinctObservatoriesPerNeoIdDF.createOrReplaceTempView("distinctObservatoriesPerNeoId")

In [ ]:
# First, compute the list of NEOIDs with >= 10 distinct observatories starting from 2023 (inner query).
# Then, keep only the most relevant NEOs that are not in that set (i.e, discard those with count >= 10.
# The kept most relevant NEOs are those of interest.
neoIdsOfInterestDF = spark.sql("""SELECT NEOID """
                               + """FROM mostRelNEOs """
                               + """WHERE NEOID NOT IN """
                               + """      (SELECT NEOID """
                               + """       FROM distinctObservatoriesPerNeoId """
                               + """       GROUP BY NEOID """
                               + """       HAVING COUNT(*)>=10)""")
        
# Register the temporary table neoIdsOfInterest
neoIdsOfInterestDF.createOrReplaceTempView("neoIdsOfInterest")

In [ ]:
# Register a function that given a null value returns the string "NONE"

def funNone(ObsId):
    if (ObsId is None):
        ObsId = "NONE"
        
    return ObsId
    

spark.udf.register("FUNCNONE",\
                     funNone, \
                     StringType())

In [ ]:
# Start from the observations made starting from 2023 by distinct observatories for each NEOID,
# and use an outerJoin to keep only the entries which satisfies the two conditions
# - most relevant NEOID
# - number of distinct observatories <= 10 (0 included!)
# missing values (i.e., no observations) are filled with "None"

res2DF = spark.sql("""SELECT neoIdsOfInterest.NEOID, FUNCNONE(ObservatoryID) """
                   + """FROM distinctObservatoriesPerNeoId RIGHT OUTER JOIN neoIdsOfInterest """
                   + "ON distinctObservatoriesPerNeoId.NEOID=neoIdsOfInterest.NEOID""")

In [ ]:
res2DF.write\
    .format("csv")\
    .option("header", False)\
    .save(outputPath2)